In [10]:
!pip install -U googlemaps

Requirement already up-to-date: googlemaps in /usr/local/lib/python3.6/dist-packages (4.2.0)


In [0]:
import googlemaps
from geopy.distance import geodesic
gmaps = googlemaps.Client(key='<put your api key>')

In [0]:

def makeOrder(address,order=[]):
  lowest=float("inf")
  selected=address
  order.append(address)
  start_location = gmaps.geocode(address)
  you = (str(start_location[0]['geometry']['location']['lat'])+","+str(start_location[0]['geometry']['location']['lng']))
  for midpoint in cluster:
    if(midpoint not in order):
      end_location = gmaps.geocode(midpoint)
      me = (str(end_location[0]['geometry']['location']['lat'])+","+str(end_location[0]['geometry']['location']['lng']))
      miles = geodesic(me,you).miles
      if(miles < lowest):
        lowest = miles
        selected = midpoint
  if(len(order) < len(cluster)):
    makeOrder(selected,order) 
  return order

In [0]:
def sortClusters(midpoint,miniCluster):
    start_location = gmaps.geocode(midpoint)
    you = (str(start_location[0]['geometry']['location']['lat'])+","+str(start_location[0]['geometry']['location']['lng']))
    mile=[]
    for points in miniCluster:
        end_location = gmaps.geocode(points)
        me = (str(end_location[0]['geometry']['location']['lat'])+","+str(end_location[0]['geometry']['location']['lng']))
        miles = geodesic(me,you).miles
        mile.append(miles)
    sorted_order=[x for _,x in sorted(zip(mile,miniCluster))]
    return list(sorted_order)

In [0]:
def calculateMiles(file):
    f=open(file,'r')
    total_miles=0
    previous=""
    previous_position=""
    current_position=""
    current=""
    for i in f.readlines():
        current=gmaps.geocode(i)
        current_position = (str(current[0]['geometry']['location']['lat'])+","+str(current[0]['geometry']['location']['lng']))
        if(previous != ""):
            miles = geodesic(previous_position,current_position).miles
            total_miles +=miles
        previous = current
        previous_position=current_position
    return(total_miles)



In [0]:
cluster={}
f=open("address.txt",'r')
for addresses in f.readlines():
  flag=0
  if cluster == {}:
    flag=1
    cluster[addresses]=[]
  else:
    for i in cluster:
      current_location = gmaps.geocode(addresses)
      midpoint = gmaps.geocode(i)
      you = (str(current_location[0]['geometry']['location']['lat'])+","+str(current_location[0]['geometry']['location']['lng']))
      me = (str(midpoint[0]['geometry']['location']['lat'])+","+str(midpoint[0]['geometry']['location']['lng']))
      miles = geodesic(me,you).miles
      if(miles <= 1.0):
        cluster[i].append(addresses)
        flag=1
        break
  if(flag == 0):
      cluster[addresses]=[]

In [17]:
p=makeOrder(list(cluster.keys())[0],[])
for c in p:
    cluster[c]=sortClusters(c,list(cluster[c]))
final_list=[]
for c in p:
    final_list.append(c)
    for i in cluster[c]:
        final_list.append(i)
out=open("result.txt","w")
for line in final_list:
    out.write(line)
out.close()

print("we have reduced miles from "+str(calculateMiles("address.txt"))+" miles to "+ str(calculateMiles("result.txt"))+" miles")

we have reduced miles from 472.3763569363071 miles to 121.81393022374365 miles
